# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
1. Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
2. Import Yelp business data and join to inspection data    
4. Join the Yelp business and Inspection Data with the Yelp Review Data
    + Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    +  Aggregate any review "count" features using this same logic
    + Combine the review documents for a restaurant into a CLOB using the same logic
    
### TO DO:
1. Determine how we want to do Levenshtein matching to combine the datasets (see the *Join Yelp Review Data with Inspection Dataset* below) 
2. Create additional engineered features
3. n-gram extraction
4. vectorization of n-grams
5. model selection
    + training
    + validaiton
    + evaluation
    +repeat with additional or removed features and data segmentation (if aggregating the review text does not yield successful results)

## Import and Clean Data

In [1]:
import psycopg2 as psy
import pandas as pd
import re
import numpy as np

In [2]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()

In [3]:
#cur.execute("SELECT * FROM public.restaurants LIMIT 50 ")
#biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_checkins LIMIT 50")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_reviews LIMIT 50")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [4]:
reviews.columns = ['bizID','reviewID','userID','type','stars','text','useful','funny','cool','date']
#get total reviews per biz
rev = reviews['bizID'].value_counts()
rev_counts = pd.DataFrame(rev).reset_index()
rev_counts.columns = ['bizID','all_review_count']

In [5]:
#not using this - can delete

checkins.columns = ['bizID','type','datetime']
#get total checkins per biz
chks  = checkins['bizID'].value_counts()
chk_counts = pd.DataFrame(chks).reset_index()
chk_counts.columns = ['bizID','checkin_counts']

### Join Yelp Review Data with Inspection Dataset

We have a few options here. While it is optimal to have as much done in Python as possible, the matching process in python is impractically slow. We can 1) use a manual implementation of Levenshtein Distance (LD), 2) use a package with optimized LD code, or 3) do the joining of the inspeciton data and review data in our database. OPtions 1& 2 are shown below, and code to retrieve the results of option 3 are at the bottom of this section.

If we decide to go with option 3, the last steps are to combine all records where the business ID, last inspection date, and inspection date are equal in order to get to one observation per restaurant-inspection combination. We can then add in any other engineered features.

**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

#### Levenshtein Option #3 (in-database) 
This option joins the yelp restaurant informaiton to each inspection record where:
 * The Levenshetein distance of the restaurant name from the two datasets is <3
 * The distance of the address from each dataset is <4
 * The date of the review is greater than the prior inspection date
 * The date of the review is less than or equal to inspeciton date on the record
 
Whitespace at the beginning and end of the name and address in each dataset is trimmed, and the strings are converted to uppercase before matching. The mathcing thresholds can be adjusted to increase potential for matching, or decrease false matches.

In [6]:
# The materialized view of the restaurant, inspection, and review data is "toronto_all"
cur.execute("SELECT * FROM public.toronto_all where review_date is not null limit 250" )
obs = pd.DataFrame(cur.fetchall())
obs.head()
obs.columns=['bizID','name','address','postal_code','neighborhood','lat','long','categories','attributes','is_open','review_cnt','hours','stars','setablishment_id','establishment_name','establishment_address','inspection_date','last_inspection','count_minor','count_sig','count_crucial','count_na','count_crucial_signficant','review_id','user_id','review_stars','review_text','useful','funny','cool','review_dt']
obs.head()

,bizID,name,address,postal_code,neighborhood,lat,long,categories,attributes,is_open,...,count_na,count_crucial_signficant,review_id,user_id,review_stars,review_text,useful,funny,cool,review_dt
0,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,OriPsMQx1cRyE9hgNnhQFg,m7z-tX6XDZ27xGhGjnI21w,5,"""I am confident when I say this place is hands...",4,0,4,2016-02-15
1,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,1WON5dUarKdWJ6-yq7Ne6g,UrfdzamoBt0WW9Ifqy7RIw,5,"The short version of my review is this: ""Hashi...",3,0,1,2015-11-18
2,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,TNpm4Hs6x2yJvYbUZSeA4w,5KbUkX5DHGtDSmqdG5LLhw,5,"""It's a great cultural experience with great f...",0,0,0,2016-03-14
3,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0pS7e898Z5AywcZQ9_DX1Q,xspGyCnzmZgsP-VUnQ3K4A,5,"""This was definitely where quality of food and...",1,0,1,2016-03-15
4,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,ePOLGcC5yJbD_DzOLfyKbA,_dRcIdWjks0phgfiY27AFQ,5,My sister and I enjoyed the lunchtime Kaiseki ...,0,0,0,2016-05-01


In [7]:
obs['bizID-dt'] = obs['bizID'] + "-" + obs['inspection_date'].map(str)

In [8]:
in_scope_rev = obs['bizID-dt'].value_counts()
in_scope_reviews = pd.DataFrame(in_scope_rev).reset_index()
in_scope_reviews.columns = ['bizID-dt','count_reviews_in_scope']
in_scope_reviews.head()

,bizID-dt,count_reviews_in_scope
0,02BXFKzu1rgaYulNGYvi6g-2016-08-23,21
1,02BXFKzu1rgaYulNGYvi6g-2015-10-08,20
2,02BXFKzu1rgaYulNGYvi6g-2016-04-01,17
3,02BXFKzu1rgaYulNGYvi6g-2016-01-19,15
4,04HfzvvANCF3hd-yx6ZACw-2016-03-22,11


In [9]:
#get dummies for star rating column
obs = pd.concat([obs, pd.get_dummies(obs['review_stars'], prefix='stars')], axis=1)
obs.head()

,bizID,name,address,postal_code,neighborhood,lat,long,categories,attributes,is_open,...,useful,funny,cool,review_dt,bizID-dt,stars_1,stars_2,stars_3,stars_4,stars_5
0,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,4,0,4,2016-02-15,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
1,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,3,0,1,2015-11-18,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
2,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0,2016-03-14,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
3,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,1,0,1,2016-03-15,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
4,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0,2016-05-01,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,0,0,0,0,1


In [10]:
stars = obs.groupby('bizID-dt')[['stars_1', 'stars_2','stars_3','stars_4','stars_5']].sum().reset_index()
stars.head()

,bizID-dt,stars_1,stars_2,stars_3,stars_4,stars_5
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,4
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,0,0,0,0,1
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,1,0,0,0,0
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,0,1,0,0,1
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,2,2,7,5,4


In [11]:
combined_revs = obs.groupby('bizID-dt')['review_text'].apply(' '.join).reset_index()
combined_revs.head()

,bizID-dt,review_text
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,"""I am confident when I say this place is hands..."
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,My sister and I enjoyed the lunchtime Kaiseki ...
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,"""The worst treatment in my entire life.\n\nThe..."
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,"""Stopped by while shopping in the eaton centre..."
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,"""I live 5 minutes away from this restaurant an..."


In [12]:
users = obs.groupby('bizID-dt')['user_id'].count().reset_index()
users.columns = ['bizID-dt','count_unique_users']
users.head()

,bizID-dt,count_unique_users
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,4
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,1
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,1
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,2
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,20


In [13]:
sub = obs[['bizID-dt','bizID','name','postal_code','lat','long','categories','attributes','is_open']]
sub = sub.drop_duplicates()
sub.head()

,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,is_open
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1
4,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1
5,01nRNgH_ukm8E2td9TTZDA-2016-05-09,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1
6,01nRNgH_ukm8E2td9TTZDA-2016-10-21,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1
8,02BXFKzu1rgaYulNGYvi6g-2015-10-08,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1


In [64]:
#merge in all data into one df
df1 = pd.merge(sub,stars,on='bizID-dt', how='left')
df2 = pd.merge(df1,combined_revs,on='bizID-dt', how='left')
df3 = pd.merge(df2,rev_counts,on='bizID', how='left')
df4 = pd.merge(df3,in_scope_reviews,on='bizID-dt', how='left')
df5 = pd.merge(df4,users,on='bizID-dt', how='left')

#make sure each bizID-dt is only appearing once in the data
print('Max of number of unique bizID-dt in df (should be 1):',max(df5['bizID-dt'].value_counts()))
df5.head()

Max of number of unique bizID-dt in df (should be 1): 1


,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,is_open,stars_1,stars_2,stars_3,stars_4,stars_5,review_text,all_review_count,count_reviews_in_scope,count_unique_users
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0,0,0,0,4,"""I am confident when I say this place is hands...",NaN,4,4
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0,0,0,0,1,My sister and I enjoyed the lunchtime Kaiseki ...,NaN,1,1
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1,1,0,0,0,0,"""The worst treatment in my entire life.\n\nThe...",NaN,1,1
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1,0,1,0,0,1,"""Stopped by while shopping in the eaton centre...",NaN,2,2
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,2,2,7,5,4,"""I live 5 minutes away from this restaurant an...",NaN,20,20


#turn cell from markdown to code

from nltk.corpus import stopwords
import nltk
stop = stopwords.words('english')
df['review_text']=df['review_text'].str.lower()
df['review_text'] = df['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
ps = nltk.stem.WordNetLemmatizer()
df['review_text']=df["review_text"].apply(lambda x:[ps.lemmatize(y,pos='v') for y in x.split()])
df['review_text']=df['review_text'].apply(lambda x: ', '.join(x))
df['review_text'] = df['review_text'].str.replace('[^\w\s]','')
df.head()

In [34]:
import ast

In [65]:
t = []
for i in range(len(df5['categories'])):
    x = ast.literal_eval(df5['categories'][i])
    t.append(x)
    
cats = pd.DataFrame(t)
cats_df = pd.get_dummies(cats, prefix='Category')

In [66]:
df = pd.concat([df5, cats_df], axis=1)
df

,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,is_open,stars_1,...,Category_Event Planning & Services,Category_Food,Category_Tapas/Small Plates,Category_Vietnamese,Category_Bars,Category_Gluten-Free,Category_Russian,Category_Ice Cream & Frozen Yogurt,Category_Nightlife,Category_Italian
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0,...,0,0,0,0,0,0,0,0,0,0
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0,...,0,0,0,0,0,0,0,0,0,0
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1,1,...,0,0,0,0,0,0,0,0,0,0
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1,0,...,0,0,0,0,0,0,0,0,0,0
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,2,...,0,0,0,0,0,0,0,0,0,0
5,02BXFKzu1rgaYulNGYvi6g-2016-01-19,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,1,...,0,0,0,0,0,0,0,0,0,0
6,02BXFKzu1rgaYulNGYvi6g-2016-12-28,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0,...,0,0,0,0,0,0,0,0,0,0
7,02BXFKzu1rgaYulNGYvi6g-2016-04-01,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,4,...,0,0,0,0,0,0,0,0,0,0
8,02BXFKzu1rgaYulNGYvi6g-2017-03-20,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,1,...,0,0,0,0,0,0,0,0,0,0
9,02BXFKzu1rgaYulNGYvi6g-2016-08-23,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,7,...,0,0,0,0,0,0,0,0,0,0
